In [1]:
import pandas as pd
import numpy as np
import re
import time

from datetime import datetime as dt

### Preprocessing

In [218]:
data = pd.read_csv("bank_transactions.csv")

In [219]:
data.dropna(inplace=True)

In [220]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1041614 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1041614 non-null  object 
 1   CustomerID               1041614 non-null  object 
 2   CustomerDOB              1041614 non-null  object 
 3   CustGender               1041614 non-null  object 
 4   CustLocation             1041614 non-null  object 
 5   CustAccountBalance       1041614 non-null  float64
 6   TransactionDate          1041614 non-null  object 
 7   TransactionTime          1041614 non-null  int64  
 8   TransactionAmount (INR)  1041614 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 79.5+ MB


In [5]:
data.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


### I don't need all the columns in this datam

In [221]:
def cleaning(text):
    text = re.sub(r'[^\w\s]','',text)
    tokens = text.lower()
    tokens = tokens.split()
    return tokens

In [222]:
data.TransactionTime = data.TransactionTime.apply(lambda x: dt.strptime(str(x).zfill(6), '%H%M%S').hour)

In [223]:
data.CustomerDOB = pd.to_datetime(data.CustomerDOB).dt.to_period('M')

In [224]:
data.TransactionDate = pd.to_datetime(data.TransactionDate).dt.to_period('M')

In [225]:
data = data.iloc[:, 2:]

## Place the numeric variable in a range of values

In [226]:
data.rename(columns = {"TransactionAmount (INR)": "TransactionAmount_INR"}, inplace = True)

In [227]:
n1 = data.CustAccountBalance.max()/100
n2 = data.TransactionAmount_INR.max()/100

In [138]:
data.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


In [228]:
a = np.quantile(data.CustAccountBalance, np.arange(0,1,0.1))
#, axis=None, out=None, overwrite_input=False, method='linear', keepdims=False, *, interpolation=None)

In [229]:
a = a.tolist()

In [230]:
a = list(map(int, a))

In [231]:
a.append(data.CustAccountBalance.max())

In [232]:
a = list(map(int, a))

In [233]:
a.remove(0)

In [234]:
a

[490, 3096, 6521, 10810, 16768, 26292, 43198, 79703, 197787, 115035495]

In [235]:
bins[0].unique()

array([0, 9, 5, 8, 6, 2, 3, 4, 7, 1], dtype=int64)

In [236]:
bins = pd.qcut(data.CustAccountBalance, 10, retbins = True, duplicates = 'drop',labels=False)


In [237]:
data.CustAccountBalance = bins[0]

In [238]:
data.CustAccountBalance = data.CustAccountBalance.apply(lambda x :  a[x])

In [239]:
data.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount_INR
0,1994-10,F,JAMSHEDPUR,26292,2016-02,14,25.0
1,2057-04,M,JHAJJAR,3096,2016-02,14,27999.0
2,1996-11,F,MUMBAI,26292,2016-02,14,459.0
3,1973-09,F,MUMBAI,115035495,2016-02,14,2060.0
4,1988-03,F,NAVI MUMBAI,10810,2016-02,18,1762.5


In [240]:
bins = pd.qcut(data.TransactionAmount_INR, 10, retbins = True, duplicates = 'drop',labels=False)


In [241]:
a = np.quantile(data.TransactionAmount_INR, np.arange(0,1,0.1))

In [242]:
a = list(map(int, a))


In [243]:
a.remove(0)

In [244]:
a.append(data.TransactionAmount_INR.max())

In [245]:
data.TransactionAmount_INR = bins[0]

In [246]:
data.TransactionAmount_INR = data.TransactionAmount_INR.apply(lambda x :  a[x])

In [247]:
data.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount_INR
0,1994-10,F,JAMSHEDPUR,26292,2016-02,14,50.00
1,2057-04,M,JHAJJAR,3096,2016-02,14,1560034.99
2,1996-11,F,MUMBAI,26292,2016-02,14,631.00
3,1973-09,F,MUMBAI,115035495,2016-02,14,2925.00
4,1988-03,F,NAVI MUMBAI,10810,2016-02,18,2925.00


let's create a new column with the joined string of all previous columns

In [248]:
data['joined_data'] = data[data.columns[:]].apply(
    lambda x: ' '.join(x.astype(str)),
    axis=1
)
#Time that it took: 
#around 4m

In [249]:
#Clean the data that i'm going to use
data["joined_data"] = data["joined_data"].apply(lambda x: cleaning(x))


In [250]:
data.head()

,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount_INR,joined_data
0,1994-10,F,JAMSHEDPUR,26292,2016-02,14,50.00,"[199410, f, jamshedpur, 26292, 201602, 14, 500]"
1,2057-04,M,JHAJJAR,3096,2016-02,14,1560034.99,"[205704, m, jhajjar, 3096, 201602, 14, 156003499]"
2,1996-11,F,MUMBAI,26292,2016-02,14,631.00,"[199611, f, mumbai, 26292, 201602, 14, 6310]"
3,1973-09,F,MUMBAI,115035495,2016-02,14,2925.00,"[197309, f, mumbai, 115035495, 201602, 14, 29250]"
4,1988-03,F,NAVI MUMBAI,10810,2016-02,18,2925.00,"[198803, f, navi, mumbai, 10810, 201602, 18, 2..."


In [252]:
prova = data.joined_data.tolist()
prova2 = set()
for x  in prova:
    for y in x:
        prova2.add(y)


In [253]:
len(prova2)

7634

In [254]:
string_data = [" ".join(lst) for lst in data.joined_data]

In [255]:
from sklearn.feature_extraction.text import CountVectorizer


In [256]:
vectorizer = CountVectorizer(binary=True)
vectorizer.fit(string_data)

CountVectorizer(binary=True)

In [257]:
matrix = vectorizer.transform(string_data)

In [258]:
from sklearn.preprocessing import binarize

In [259]:
x = binarize(matrix) 

In [260]:
matrix.astype(np.int8)

<1041614x7601 sparse matrix of type '<class 'numpy.int8'>'
	with 6352539 stored elements in Compressed Sparse Row format>

In [261]:
matrix.shape



(1041614, 7601)

In [262]:
matrix_tran = matrix.transpose()
matrix_tran.shape

(7601, 1041614)

In [263]:
from numpy import argmax

In [318]:
def findSignatureLine(matrix):
    first1 = [argmax(col) for col in matrix]
    return first1

In [319]:
from sklearn.utils import shuffle

def createSignatureMatrix(iter, matrix):
    signature_list_matrix = []
    new_m = matrix
    for x in range(iter):
        new_m = shuffle(matrix)
        signature_list_matrix.append(findSignatureLine(new_m))
    return pd.DataFrame(signature_list_matrix)
    

In [320]:
signature = createSignatureMatrix(2, matrix)

In [321]:
signature.head()

,0,1,2,3,4,5,6,7,8,9,...,1041604,1041605,1041606,1041607,1041608,1041609,1041610,1041611,1041612,1041613
0,18,18,18,9,4,3,36,36,3,40,...,38,3,3,23,21,3,38,3,3,21
1,18,18,18,9,4,3,36,36,3,40,...,38,3,3,23,21,3,38,3,3,21
